In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

ksprojects = drive.CreateFile({'id':'1GB1DzHU5KoLdAAOyOtkd4y4S2sR4fjk4'}) 
ksprojects.GetContentFile('ksprojects.csv')  
df = pd.read_csv('ksprojects.csv')


In [2]:
# How many rows data are in the DataFrame? There are 65,418 rows.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65418 entries, 0 to 65417
Data columns (total 13 columns):
id               65418 non-null int64
name             65418 non-null object
category         65418 non-null object
main_category    65418 non-null object
deadline         65418 non-null object
goal             65418 non-null float64
launched         65418 non-null object
state            65418 non-null object
backers          65418 non-null int64
country          65418 non-null object
usd_pledged      64673 non-null float64
currency         65418 non-null object
pledged          65418 non-null float64
dtypes: float64(3), int64(2), object(8)
memory usage: 6.5+ MB


In [3]:
# What are the names and data types of the columns? The 13 column names and types are listed below.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65418 entries, 0 to 65417
Data columns (total 13 columns):
id               65418 non-null int64
name             65418 non-null object
category         65418 non-null object
main_category    65418 non-null object
deadline         65418 non-null object
goal             65418 non-null float64
launched         65418 non-null object
state            65418 non-null object
backers          65418 non-null int64
country          65418 non-null object
usd_pledged      64673 non-null float64
currency         65418 non-null object
pledged          65418 non-null float64
dtypes: float64(3), int64(2), object(8)
memory usage: 6.5+ MB


In [4]:
# Do any of the columns contain null values? Yes, the usd_pledged column contains null values. 

df.isnull().any()

id               False
name             False
category         False
main_category    False
deadline         False
goal             False
launched         False
state            False
backers          False
country          False
usd_pledged       True
currency         False
pledged          False
dtype: bool

In [5]:
# Find all successful documentary projects and sort them by the amount pledged. Print the top 10 highest pledges.

successful_docs = (df['category'] == 'Documentary') & (df['state'] == 'successful')
successful_docs_df = df.iloc[successful_docs.values]
sorted_docs = successful_docs_df.sort_values(by = 'pledged', ascending = False)
sorted_docs.head(10)

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
11731,1078982332,BRIDEGROOM - An American Love Story,Documentary,Film & Video,2012-07-20 01:00:28,300000.0,2012-06-20 01:00:28,successful,6508,US,384376.0,USD,384376.0
24166,1162836718,"""The Wrecking Crew"" The Untold Story of Rock &...",Documentary,Film & Video,2013-12-21 08:59:00,250000.0,2013-10-30 21:06:34,successful,4245,US,313158.0,USD,313158.0
12127,1326363492,The Culture High,Documentary,Film & Video,2012-08-04 04:00:00,190000.0,2012-06-22 01:40:20,successful,3448,US,240022.0,USD,240022.0
61248,1203470139,Curious George Documentary,Documentary,Film & Video,2016-08-25 00:00:00,175000.0,2016-07-26 18:01:41,successful,1483,US,2028.0,USD,186010.0
28003,1205913248,ILEANA'S SMILE,Documentary,Film & Video,2014-05-21 09:05:00,133000.0,2014-04-21 09:05:00,successful,1173,US,177421.0,USD,177421.0
27402,1135999291,Rewind To Fast-Forward,Documentary,Film & Video,2014-05-01 07:59:00,137000.0,2014-03-30 13:35:31,successful,4395,US,176746.0,USD,176746.0
16243,1061853312,Linsanity: the Movie,Documentary,Film & Video,2013-02-16 00:00:00,117000.0,2013-01-15 19:46:34,successful,2102,US,167917.0,USD,167917.0
27268,1228065368,El Voto Hispano | The Documentary,Documentary,Film & Video,2014-04-27 08:15:23,158000.0,2014-03-18 07:15:23,successful,461,US,165401.0,USD,165401.0
43820,1194449879,"PlantPure Nation, a Documentary with Dr. T. Co...",Documentary,Film & Video,2015-05-14 17:41:41,150000.0,2015-03-15 16:41:41,successful,1099,US,163930.0,USD,163930.0
22237,122290911,The Killing of Tony Blair,Documentary,Film & Video,2013-10-07 18:17:14,50000.0,2013-08-28 18:17:14,successful,4459,GB,255150.0,GBP,163891.0


In [6]:
# Create a new column named average_per_backer and set its value to the total amount pledged / number of backers. 
# What happened to the rows with 0 backers? The entry became NaN.
# How can this be dealt with? I replaced the NaN values in the average_per_backer column with 0's. 

new_df = df.assign(average_per_backer = df.pledged/df.backers)
values = {'average_per_backer':0}
filled_df = new_df.fillna(value=values)
filled_df

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged,average_per_backer
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00,10.000000
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00,50.000000
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00,42.718750
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00,273.958333
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66,22.802727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65413,1107492255,DODEKA - music finally within everyone's reach!,Sound,Technology,2017-02-03 14:51:00,55000.0,2016-12-05 14:51:00,live,9,CH,1022.12,CHF,9287.00,1031.888889
65414,1373187022,World's Smallest Water-resistant Magnetic Came...,Gadgets,Technology,2017-02-03 17:23:11,2000.0,2016-12-05 17:23:11,canceled,148,US,5474.00,USD,5552.00,37.513514
65415,1212504494,Varnville Central Station - Guest House #1,Product Design,Design,2017-02-03 20:17:57,30000.0,2016-12-05 20:17:57,live,0,US,0.00,USD,0.00,0.000000
65416,1347446277,Luminaria emergencia Led con mensajes.,Technology,Technology,2017-02-03 21:51:02,30000.0,2016-12-05 21:51:02,live,0,ES,0.00,EUR,0.00,0.000000


In [7]:
# Drop all rows with 0 backers then repeat the previous exercise. **I'm not sure which previous exercise this instruction is referring to...**

drop_df = df[df.backers != 0]
another_df = drop_df.assign(average_per_backer = df.pledged/df.backers)
another_df


,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged,average_per_backer
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.000,USD,40.00,10.000000
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.000,USD,700.00,50.000000
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.000,USD,1367.00,42.718750
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.000,USD,6575.00,273.958333
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.660,USD,501.66,22.802727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65408,1166775907,The sports wall racks collection by MEOLLO™,Product Design,Design,2017-02-01 23:59:00,8000.0,2016-12-05 18:00:10,live,4,ES,153.600,EUR,144.00,36.000000
65410,1120449668,Not For You: Family Narratives of Denial & Com...,Food,Food,2017-02-03 07:24:36,3500.0,2016-12-05 07:24:36,live,24,US,235.000,USD,527.00,21.958333
65411,1105965008,Anyone can Learn English. Anywhere. For Free.,Technology,Technology,2017-02-03 07:40:46,16000.0,2016-12-05 07:40:46,live,10,GB,159.122,GBP,475.00,47.500000
65413,1107492255,DODEKA - music finally within everyone's reach!,Sound,Technology,2017-02-03 14:51:00,55000.0,2016-12-05 14:51:00,live,9,CH,1022.120,CHF,9287.00,1031.888889


In [8]:
# Create a crosstab to get a count of records for each combination of state and category.

ct_df = pd.crosstab(df['category'], df['state'])
ct_df

state,canceled,failed,live,successful,suspended,undefined
category,,,,,,
3D Printing,17,59,3,43,2,1
Academic,21,86,3,43,4,0
Accessories,48,238,16,121,2,1
Action,14,79,1,16,0,0
Animals,2,31,0,11,1,0
...,...,...,...,...,...,...
Woodworking,10,125,4,38,2,0
Workshops,0,12,1,8,1,0
World Music,31,174,2,168,1,0


In [9]:
# This data consists of information about all flights that departed NYC in 2013. Download two datasets: flights and airlines.
# Merge these two DataFrames into a single DataFrame: nycflights13.
# Are there any missing values in this DataFrame? Yes, in the dep_time, dep_delay, arr_time, arr_delay, tailnum, and air_time columns. 
# If so, drop them. **Does this mean to drop the whole row containing the missing values? Because that's what I did :) **

flights = drive.CreateFile({'id':'1fYuYPZ4KhyGLjCh0Tc5E3Go1Lqn6WrCb'}) 
flights.GetContentFile('flights.csv')  
flights_df = pd.read_csv('flights.csv')

airlines = drive.CreateFile({'id':'17PTQ1VJlP4G6S_WzmStR5AcQ2J7sC5A7'}) 
airlines.GetContentFile('airlines.csv')  
airlines_df = pd.read_csv('airlines.csv')

nycflights13 = pd.merge(flights_df, airlines_df)
nycflights13 = nycflights13.dropna()
nycflights13.isnull().any()

year              False
month             False
day               False
dep_time          False
sched_dep_time    False
dep_delay         False
arr_time          False
sched_arr_time    False
arr_delay         False
carrier           False
flight            False
tailnum           False
origin            False
dest              False
air_time          False
distance          False
hour              False
minute            False
time_hour         False
name              False
dtype: bool

In [10]:
population = drive.CreateFile({'id':'1nm7Ej_MpH8Jbh6hd311fxaBwASXZ-xep'}) 
population.GetContentFile('state-populations.csv')  
population_df = pd.read_csv('state-populations.csv')
population_df

regions = drive.CreateFile({'id':'1ATAx6h6DbkvnniBEfENzA0ckPMkK-0_g'}) 
regions.GetContentFile('census-divisions.csv')  
regions_df = pd.read_csv('census-divisions.csv')
regions_df

census = pd.merge(population_df, regions_df)
census.head()

,state,2010,2011,2012,2013,2014,2015,2016,region,division
0,Alabama,4785492,4799918,4815960,4829479,4843214,4853875,4863300,South,East South Central
1,Alaska,714031,722713,731089,736879,736705,737709,741894,West,Pacific
2,Arizona,6408312,6467163,6549634,6624617,6719993,6817565,6931071,West,Mountain
3,Arkansas,2921995,2939493,2950685,2958663,2966912,2977853,2988248,South,West South Central
4,California,37332685,37676861,38011074,38335203,38680810,38993940,39250017,West,Pacific


In [11]:
census_melt = pd.melt(census, id_vars=['state'], value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016'], var_name='year', value_name = 'population')
census_melt

,state,year,population
0,Alabama,2010,4785492
1,Alaska,2010,714031
2,Arizona,2010,6408312
3,Arkansas,2010,2921995
4,California,2010,37332685
...,...,...,...
352,Virginia,2016,8411808
353,Washington,2016,7288000
354,West Virginia,2016,1831102
355,Wisconsin,2016,5778708


In [12]:
# Group the data by year and summarize it.

census_melt.groupby('year').describe()



population                              ...                                  
          count          mean           std  ...        50%        75%         max
year                                         ...                                  
2010       51.0  6.065651e+06  6.839536e+06  ...  4348662.0  6654375.0  37332685.0
2011       51.0  6.111046e+06  6.907730e+06  ...  4369354.0  6717221.5  37676861.0
2012       51.0  6.156831e+06  6.974942e+06  ...  4384799.0  6776617.0  38011074.0
2013       51.0  6.200096e+06  7.038774e+06  ...  4400477.0  6837396.0  38335203.0
2014       51.0  6.246342e+06  7.110811e+06  ...  4413057.0  6902053.5  38680810.0
2015       51.0  6.292091e+06  7.181603e+06  ...  4424611.0  6988927.5  38993940.0
2016       51.0  6.335834e+06  7.243849e+06  ...  4436974.0  7109535.5  39250017.0

[7 rows x 8 columns]

In [13]:
# Group the data by region, division and year and summarize it.

final_census = pd.melt(census, id_vars=['state', 'region', 'division'], value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016'], var_name='year', value_name = 'population')
final_census.groupby(['region', 'division', 'year'])['population'].describe()


count        mean  ...         75%         max
region  division           year                     ...                        
Midwest East North Central 2010    5.0   9288169.4  ...  11540983.0  12841578.0
                           2011    5.0   9301433.8  ...  11544824.0  12860012.0
                           2012    5.0   9314559.0  ...  11550839.0  12870798.0
                           2013    5.0   9332093.0  ...  11570022.0  12879505.0
                           2014    5.0   9346265.8  ...  11594408.0  12867544.0
...                                ...         ...  ...         ...         ...
West    Pacific            2012    5.0  10185665.0  ...   6895226.0  38011074.0
                           2013    5.0  10274464.0  ...   6968006.0  38335203.0
                           2014    5.0  10371286.2  ...   7054196.0  38680810.0
                           2015    5.0  10468346.0  ...   7160290.0  38993940.0
                           2016    5.0  10560386.6  ...   7288000.0  39250017.0

[63 rows x 8 columns]